In [4]:
pip install pysam

     |████████████████████████████████| 10.2 MB 2.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pysam as pys
from pysam import VariantFile

In [74]:
class mutations_per_chromozom:
    def __init__(self, chromozom, positions):
        self.chromozom = chromozom
        self.positions = positions
    

def extract_predictions_from_VCF (file_name):
    num = 0
    array = [] # [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22, X (23), Y (24), MT (25)]
    handle = VariantFile(file_name)
    index  = 1
    
    array.append(0)
    for i in range (1,26):
        array.append(mutations_per_chromozom(i,[]))
    
    # Calculate only SNPs
    for record in handle.fetch():
        allels = record.alleles
        if (len(allels) == 2):
            ref, alt = record.alleles
            if(len(ref) == 1 and len(alt) == 1): 
                num = num + 1
                if (record.chrom == 'X'):
                    array[23].positions.append(record.pos)
                elif(record.chrom == 'Y'):
                    array[24].positions.append(record.pos)
                elif(record.chrom == 'MT'):
                    array[25].positions.append(record.pos)
                else:
                     array[int(record.chrom)].positions.append(record.pos)

    # returning predictions number and all prediction posssitions in array.
    return num, array 


In [75]:
# count number of predictions in VCF truth set (gatk_haplotype_caller)

gatk_vcf = "/sbgenomics/workspace/output.vcf"
gatk_predictions_num, gatk_predictions_array = extract_predictions_from_VCF(gatk_vcf)  
print ("Number of predictions in VCF truth set is: ",gatk_predictions_num)

Number of predictions in VCF truth set is:  62295


In [76]:
# count number of predictions in VCF test set (freebayes)

freebayes_vcf = "/sbgenomics/workspace/var.vcf"
freebayes_predictions_num, freebayes_predictions_array = extract_predictions_from_VCF(freebayes_vcf)  
print ("Number of predictions in VCF truth set is: ", freebayes_predictions_num)

Number of predictions in VCF truth set is:  146717


In [77]:
# Calculating number of True/False positives.
# VCF generated by GATK is truth set and VCF generated by freebayes is test set
# True positive is number of predictions which exist in both VCF files.
# Freebayes is test set, so we take prediction from it and then check do that prediction exist in GATK VCF.
# If prediction (mutation) exist in gatk that is TRUE POSITIVE, otherwise is FALSE POSITIVE

# We need to callculate those values for each chromozom and then we will add them.

def calculate_true_and_false_positive_values (truth_set_array, test_set_array):
    true_positive = 0
    false_positive = 0
    for chrom in range (1,26): # 1..25
        for position in test_set_array[chrom].positions:
            if position in truth_set_array[chrom].positions:
                true_positive = true_positive + 1
            else:
                false_positive = false_positive + 1
     
    return true_positive, false_positive

true, false = calculate_true_and_false_positive_values(gatk_predictions_array, freebayes_predictions_array)

print ("Number of TRUE POSITIVES  := ", true)
print ("Number of FALSE POSITIVES := ", false)

Number of TRUE POSITIVES  :=  58578
Number of FALSE POSITIVES :=  88139


In [78]:
# We can see that number of TRUE_POSITIVES is just a little bit lower from number of predictions in GATK
# Important thing here is that number of TRUE_POSITIVES + FALSE_POSITIVES is exactly the same  as number 
# of predictions in freebayes VCF file.

print("TRUE_POSITIVES + FALSE_POSITIVES := ", true + false )

TRUE_POSITIVES + FALSE_POSITIVES :=  146717


In [79]:
# On similar way we can calculate number of FALSE NEGATIVES
# as number of predictions which are existing in GATK but not in FREEBAYES

def calculate_false_negative_values (truth_set_array, test_set_array):
    false_negative = 0
    for chrom in range (1,26): # 1..25
        for position in truth_set_array[chrom].positions:
            if not position in test_set_array[chrom].positions:
                false_negative = false_negative + 1
     
    return false_negative

false_neg = calculate_false_negative_values (gatk_predictions_array, freebayes_predictions_array)

print ("FALSE_NEGATIVES := ", false_neg)

FALSE_NEGATIVES :=  3717


In [80]:
# One thing to note is that sum of FALSE_NEGATIVES and TRUE POSITIVES is same as number of predictions in GATK VCF

print ("Number of FALSE_NEGATIVES + TRUE POSITIVES := ", true + false_neg)

Number of FALSE_NEGATIVES + TRUE POSITIVES :=  62295


In [84]:
# ALL CALCULATED DATA:

print ("ALL CALCULATED DATA (SNPs only): \n")
print ("Number of TRUE POSITIVES  := ", true)
print ("Number of FALSE POSITIVES := ", false)
print ("Number of FALSE_NEGATIVES := ", false_neg)

ALL CALCULATED DATA (SNPs only): 

Number of TRUE POSITIVES  :=  58578
Number of FALSE POSITIVES :=  88139
Number of FALSE_NEGATIVES :=  3717


In [85]:
## DATA GENERATED BY TOOL
## METRICS CLASS    picard.vcf.GenotypeConcordanceContingencyMetricsVARIANT_TYPE    
##      TRUTH_SAMPLE    CALL_SAMPLE    TP_COUNT    TN_COUNT    FP_COUNT    FN_COUNT    EMPTY_COUNT
## SNP    HCC1143        HCC1143        58372       28546       6620        2795           3
## INDEL  HCC1143        HCC1143        3722        4851        5033        4163           52